
### Dataset

In [1]:
env = 'Breakout'
num_episodes = 25 #use them all?

In [2]:
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu
import numpy as np

def distance(model, episode):
    z, d = sssn.distance(model, episode)
    return tu.to_numpy(z), tu.to_numpy(d)
    
def plot_latent(model, episode):
    z = tu.to_numpy(sssn.encode(model, episode))
    print(z)
    images = T.HWC(tu.to_numpy(episode))
    return J.scatter_image(z[:,0], z[:,1], images, scatter_colour='blue', line_colour='#b9d1fa', scale=1.5)

def order_by(model, episode):
    d, z = distance(model, episode)
    images = T.HWC(tu.to_numpy(episode))
    i = np.argsort(-d)
    return d[i], images[i]

Failed to initialise kaggle.
USING DEVICE: cpu
aad
aad.clean
aad.clean.BeamRider
aad.clean.BeamRider.action
aad.clean.BeamRider.state
aad.clean.Breakout
aad.clean.Breakout.action
aad.clean.Breakout.state
aad.clean.Enduro
aad.clean.Enduro.action
aad.clean.Enduro.state
aad.clean.Pong
aad.clean.Pong.action
aad.clean.Pong.state
aad.clean.Qbert
aad.clean.Qbert.action
aad.clean.Qbert.state
aad.clean.Seaquest
aad.clean.Seaquest.action
aad.clean.Seaquest.state
aad.clean.SpaceInvaders
aad.clean.SpaceInvaders.action
aad.clean.SpaceInvaders.state
aad.raw
aad.raw.BeamRider
aad.raw.BeamRider.action
aad.raw.BeamRider.state
aad.raw.Breakout
aad.raw.Breakout.action
aad.raw.Breakout.state
aad.raw.Enduro
aad.raw.Enduro.action
aad.raw.Enduro.state
aad.raw.Pong
aad.raw.Pong.action
aad.raw.Pong.state
aad.raw.Qbert
aad.raw.Qbert.action
aad.raw.Qbert.state
aad.raw.Seaquest
aad.raw.Seaquest.action
aad.raw.Seaquest.state
aad.raw.SpaceInvaders
aad.raw.SpaceInvaders.action
aad.raw.SpaceInvaders.state
aad.anomaly

In [3]:
import datasets
from pprint import pprint
import numpy as np

dataset = datasets.dataset('aad.raw.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
episodes = [x for x in dataset.state.load(num_episodes)]
for i,e in enumerate(episodes):
    print("episode:", i, e.shape)
    
print(np.sum([e.shape[0] for e in episodes]))
    
episode_test = episodes[-1]
episodes = episodes[:-1]


loading dataset aad.raw.Breakout.state...


IntProgress(value=0, max=25)

episode: 0 torch.Size([2867, 3, 210, 160])
episode: 1 torch.Size([2359, 3, 210, 160])
episode: 2 torch.Size([4901, 3, 210, 160])
episode: 3 torch.Size([2200, 3, 210, 160])
episode: 4 torch.Size([2236, 3, 210, 160])
episode: 5 torch.Size([2022, 3, 210, 160])
episode: 6 torch.Size([2461, 3, 210, 160])
episode: 7 torch.Size([3422, 3, 210, 160])
episode: 8 torch.Size([2171, 3, 210, 160])
episode: 9 torch.Size([2036, 3, 210, 160])
episode: 10 torch.Size([2589, 3, 210, 160])
episode: 11 torch.Size([2343, 3, 210, 160])
episode: 12 torch.Size([1767, 3, 210, 160])
episode: 13 torch.Size([2228, 3, 210, 160])
episode: 14 torch.Size([1772, 3, 210, 160])
episode: 15 torch.Size([2546, 3, 210, 160])
episode: 16 torch.Size([2984, 3, 210, 160])
episode: 17 torch.Size([1329, 3, 210, 160])
episode: 18 torch.Size([2441, 3, 210, 160])
episode: 19 torch.Size([1783, 3, 210, 160])
episode: 20 torch.Size([2105, 3, 210, 160])
episode: 21 torch.Size([2239, 3, 210, 160])
episode: 22 torch.Size([1946, 3, 210, 160]

# Train


In [ ]:
from types import SimpleNamespace

import pyworld.toolkit.tools.wbutils as wbu
import pyworld.toolkit.tools.fileutils as fu
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.visutils.jupyter as J

import anomapy.train.sssn as sssn

CONFIG = sssn.default_config()
CONFIG.update(dataset.meta.to_dict())
CONFIG['latent_shape'] = 256
CONFIG['epochs'] = 1
CONFIG['batch_size'] = 128
CONFIG['total_states'] = sum([e.shape[0] for e in episodes])
CONFIG['episodes'] = len(episodes)
CONFIG = SimpleNamespace(**CONFIG)

print(CONFIG)

DRYRUN = True
PROJECT = "anomaly-detection"
RUN_ID = "{0}-{1}-{2}".format(CONFIG.model, CONFIG.name, fu.file_datetime())
RUN_TAGS = [CONFIG.model, CONFIG.name]

wbu.dryrun(DRYRUN)

optimiser = sssn.new(dryrun=DRYRUN, **CONFIG.__dict__)
model = optimiser.model

wb = wbu.WB(PROJECT, model, id=RUN_ID, tags=RUN_TAGS, config=CONFIG.__dict__)
#loss_plot = J.dynamic_plot(update_after=10)

#z = tu.to_numpy(sssn.encode(model, episodes[0]))
#plot = J.plot(z[:,0], z[:,1], mode=J.line_mode.both)

with wb:
    for i in range(CONFIG.epochs):
        print("---- epoch: ", i)
        for episode in episodes:
            for loss in sssn.epoch(optimiser, episode, CONFIG.batch_size):
                pass #loss_plot.update(None, loss['loss'])
            #z = tu.to_numpy(sssn.encode(model, episodes[0]))
            #plot.set_data(z[:,0], z[:,1])
            print("loss:", optimiser.cma())


# Load Model

In [1]:
from types import SimpleNamespace
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.wbutils as wbu


def distance(model, episode):
    z, d = sssn.distance(model, episode)
    return tu.to_numpy(z), tu.to_numpy(d)


env = "Breakout"
dryruns = sorted([r for r in wbu.dryruns() if env in r])
print(dryruns[-1])

models, config = wbu.load(dryruns[-1]) #load the most recently trained model
model = models['model.pt'].load(sssn.model(**config))
config = SimpleNamespace(**config)

Failed to initialise kaggle.
USING DEVICE: cpu
aad
aad.clean
aad.clean.BeamRider
aad.clean.BeamRider.action
aad.clean.BeamRider.state
aad.clean.Breakout
aad.clean.Breakout.action
aad.clean.Breakout.state
aad.clean.Enduro
aad.clean.Enduro.action
aad.clean.Enduro.state
aad.clean.Pong
aad.clean.Pong.action
aad.clean.Pong.state
aad.clean.Qbert
aad.clean.Qbert.action
aad.clean.Qbert.state
aad.clean.Seaquest
aad.clean.Seaquest.action
aad.clean.Seaquest.state
aad.clean.SpaceInvaders
aad.clean.SpaceInvaders.action
aad.clean.SpaceInvaders.state
aad.raw
aad.raw.BeamRider
aad.raw.BeamRider.action
aad.raw.BeamRider.state
aad.raw.Breakout
aad.raw.Breakout.action
aad.raw.Breakout.state
aad.raw.Enduro
aad.raw.Enduro.action
aad.raw.Enduro.state
aad.raw.Pong
aad.raw.Pong.action
aad.raw.Pong.state
aad.raw.Qbert
aad.raw.Qbert.action
aad.raw.Qbert.state
aad.raw.Seaquest
aad.raw.Seaquest.action
aad.raw.Seaquest.state
aad.raw.SpaceInvaders
aad.raw.SpaceInvaders.action
aad.raw.SpaceInvaders.state
aad.anomaly

# Load Anomalies


In [2]:
import datasets
import numpy as np

dataset = datasets.dataset('aad.anomaly.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
anoms = [(a,e[0]) for a,e in dataset.meta.anomaly.items()]
a_episodes = [d for d in dataset.state.load_files(*[e[1] for e in anoms])]
a_labels = [d for d in dataset.label.load_files(*[e[1] for e in anoms])]
a_tlabels = [np.logical_or(l[:-1], l[1:]).astype(np.uint8) for l in a_labels] #transition labels

loading dataset aad.anomaly.Breakout.state...


IntProgress(value=0, max=6)

loading dataset aad.anomaly.Breakout.label...


IntProgress(value=0, max=6)

# ROC

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, recall_score, precision_score,f1_score
from sklearn.metrics import auc, confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score

import plotly
import plotly.graph_objects as go

import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.plot as vplot
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.fileutils as fu
import pyworld.toolkit.tools.datautils as du


import numpy as n
from pprint import pprint

def roc(label, score):
    assert label.shape[0] == score.shape[0]
    fpr, tpr, _ = roc_curve(label, score)
   
    return fpr, tpr

fprs = []
tprs = []
aucs = {}
avg_prec = {}
legend = []
for i in range(len(a_episodes[:-1])):
    episode = a_episodes[i]
    anomaly = anoms[i][0]
    label = a_tlabels[i]

    z, score = distance(model, episode)
    latent_space = tu.to_numpy(z)
    score = du.normalise(score)
    fpr, tpr, _ = roc_curve(label, score)
    aucs[anomaly] = auc(fpr, tpr)
    prec, rec, _ = precision_recall_curve(label,score)
    avg_prec[anomaly] = average_precision_score(label,score)
    fprs.append(fpr)
    tprs.append(tpr)
    legend.append(anomaly)
    print("\nEpisode", i)
    print(anomaly)
    print("\nAUC:\n",auc(fpr, tpr))
    print("\nFPR:\n",fpr)
    print("\nTPR:\n",tpr)
    #print("\nAverage Precision Scores\n",average_precision_score(label,score))
    
    #print("\nConfusion Matrix:\n",confusion_matrix(label,score))
    #print("\nRecall\n",recall_score(label,score))
    #print("\nPrecision\n",precision_score(label,score))
    #print("\nF1Score\n",f1_score(label,score))


print("\n\nAUCs\n")    
pprint(aucs)
# legend[0] = 'flicker'
# legend[1] = 'visual artefact'
# legend = [l.replace('_', ' ') for l in legend]
# print(legend)

# plot = J.plot(fprs, tprs, legend=legend, show=False)
# plot.fig.update_layout(showlegend=True, autosize=False, width=500, height=400, margin=dict(l=5,b=5,r=5,t=5))
# plot.fig.update_layout(dict(legend=dict(xanchor='center', x=0.5, orientation='h')))

#path = "/home/ben/Downloads/rocs/"
#plot.fig.write_image(path + "{0}.png".format(env))
#fu.save(path + "{0}.json".format(env), aucs)



In [6]:
from sklearn.metrics import roc_curve, roc_auc_score, recall_score, precision_score,f1_score
from sklearn.metrics import auc, confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score

import plotly
import plotly.graph_objects as go

import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.plot as vplot
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.fileutils as fu
import pyworld.toolkit.tools.datautils as du


import numpy as n
from pprint import pprint

def roc(label, score):
    assert label.shape[0] == score.shape[0]
    fpr, tpr, _ = roc_curve(label, score)
   
    return fpr, tpr

fprs = []
tprs = []
aucs = {}
avg_precs = {}
legend = []
for i in range(len(a_episodes[:-1])):
    episode = a_episodes[i]
    anomaly = anoms[i][0]
    label = a_tlabels[i]

    z, score = distance(model, episode)
    latent_space = tu.to_numpy(z)
    score = du.normalise(score)
    fpr, tpr, _ = roc_curve(label, score)
    aucs[anomaly] = auc(fpr, tpr)
    prec, rec, _ = precision_recall_curve(label,score)
    avg_precs[anomaly] = average_precision_score(label,score)
    fprs.append(fpr)
    tprs.append(tpr)
    legend.append(anomaly)
    print("\nEpisode", i)
    print(anomaly)
    print("\nAvg Precision Score:\n",avg_precs[anomaly])
    print("\nPrecision:\n",prec)
    print("\nRecall:\n",rec)
    
    #print("\nConfusion Matrix:\n",confusion_matrix(label,score))
    #print("\nRecall\n",recall_score(label,score))
    #print("\nPrecision\n",precision_score(label,score))
    #print("\nF1Score\n",f1_score(label,score))


print("\n\nAverage Precision Scores\n")    
pprint(avg_precs)
# legend[0] = 'flicker'
# legend[1] = 'visual artefact'
# legend = [l.replace('_', ' ') for l in legend]
# print(legend)

# plot = J.plot(fprs, tprs, legend=legend, show=False)
# plot.fig.update_layout(showlegend=True, autosize=False, width=500, height=400, margin=dict(l=5,b=5,r=5,t=5))
# plot.fig.update_layout(dict(legend=dict(xanchor='center', x=0.5, orientation='h')))

#path = "/home/ben/Downloads/rocs/"
#plot.fig.write_image(path + "{0}.png".format(env))
#fu.save(path + "{0}.json".format(env), aucs)




Episode 0
block

Avg Precision Score:
 0.9155208946093999

Precision:
 [0.19008264 0.19025735 0.19043238 ... 1.         1.         1.        ]

Recall:
 [1.         1.         1.         ... 0.00966184 0.00483092 0.        ]

Episode 1
flicker

Avg Precision Score:
 0.5866166247763965

Precision:
 [0.17974322 0.09803922 0.09811617 ... 1.         1.         1.        ]

Recall:
 [1.         0.49603175 0.49603175 ... 0.00793651 0.00396825 0.        ]

Episode 2
freeze

Avg Precision Score:
 0.3356803293111978

Precision:
 [0.36429872 0.05013674 0.05018248 ... 0.         0.         1.        ]

Recall:
 [1.         0.09166667 0.09166667 ... 0.         0.         0.        ]

Episode 3
freeze_skip

Avg Precision Score:
 0.3059837993465002

Precision:
 [0.27404844 0.04204753 0.042086   ... 1.         1.         1.        ]

Recall:
 [1.         0.11616162 0.11616162 ... 0.00505051 0.00252525 0.        ]

Episode 4
split_horizontal

Avg Precision Score:
 0.867548946697728

Precision:
 [0.09

In [7]:
from sklearn.metrics import auc, confusion_matrix, precision_recall_fscore_support
print("\nRecall\n",precision_recall_fscore_support(label,score))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
import tensorflow as tf

learning_rate = 0.01
training_steps = 100
batch_size = 256
num_classes = 2

# Logistic regression (Wx + b).
def logistic_regression(x):
    # Apply sigmoid to normalize the logits to a probability distribution.
    return tf.nn.sigmoid(tf.matmul(x, W) + b)

# Calculate cross entropy
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# the RMSprop optimizer with the default hyper-parameters
optimizer = tf.optimizers.legacy.RMSprop(learning_rate)

In [ ]:
# Accuracy metric.
def accuracy(y_pred, y_true):
  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.metrics import auc, confusion_matrix
np_config.enable_numpy_behavior()

# Run training for the given number of steps.
k=1
fprs = []
tprs = []
aucs = {}

for i in range(len(a_episodes[:-1])):

    episode = a_episodes[i]
    anomaly = anoms[i][0]
    label = a_tlabels[i]
    latent_space, score = distance(model, episode)

    score1 = score.reshape(score.shape[0],1)
    x = tf.concat([latent_space[1:], score1], axis=1)
    W = tf.Variable(tf.ones([x.shape[1], 1]), name="weight")
    b = tf.Variable(tf.zeros([1]), name="bias")

    for step in range(training_steps):
        # Run the optimization to update W and b values.
        run_optimization(x, label)     
        pred = logistic_regression(x)
        loss = cross_entropy(pred, label)
        accur = accuracy(pred, label)
            
    fpr, tpr, _ = roc_curve(label, pred)
    aucs[anomaly] = auc(fpr, tpr)
    fprs.append(fpr)
    tprs.append(tpr)
    #print("eposide: ",i, "loss: ",loss, "score: ",pred)
print("Auc after active learning: ",aucs)

# images = T.HWC(tu.to_numpy(episode))
# up_label = []
# up_episode = []
# top_k = np.argsort(pred)
# for j in range(k):
#     J.images(images[top_k[j]])

#     feedback = input("Is this a valid anomaly? [y/n]")
#     if feedback.lower() == 'y':
#         up_label.append(1)
#     else:
#         up_label.append(0)

    
    

# Histograms

In [ ]:
anoms[0] = ('flicker', anoms[1])
anoms[1] = ('visual artefact', anoms[1])
anoms = [(a[0].replace('_',' '), a[1]) for a in anoms]
for a in anoms:
    print(a)

In [ ]:
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.plot as vplot
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.datautils as du
import numpy as np


import plotly
import plotly.graph_objects as go


subplot = plotly.subplots.make_subplots(rows=3,cols=2, vertical_spacing = 0.08, horizontal_spacing = 0.05, subplot_titles=[a[0] for a in anoms[:-1]])



showlegend = True

def histogram(model, episode, labels, title, i, j, bins=50):
    z, d = distance(model, episode)
    d = du.normalise(d)
    
    d_a = d[labels==1]
    d_n = d[labels==0]
    print(d.shape, d_a.shape, d_n.shape)
    binsize = (np.max(d) - np.min(d)) / bins
    print("anomaly count: ", d_a.shape[0], "normal count: ", d_n.shape[0])
    subplot.add_trace(go.Histogram(x=d_a, marker=dict(color='red'), name='anomaly', showlegend=showlegend, xbins=dict(size=binsize)), row=i, col=j)
    subplot.add_trace(go.Histogram(x=d_n, marker=dict(color='blue'), name='normal', showlegend=showlegend, xbins=dict(size=binsize)), row=i, col=j)

histogram(model, a_episodes[0], a_tlabels[0], anoms[0], 1, 1)
showlegend=False
histogram(model, a_episodes[1], a_tlabels[1], anoms[1], 1, 2)
histogram(model, a_episodes[2], a_tlabels[2], anoms[2], 2, 1)
histogram(model, a_episodes[3], a_tlabels[3], anoms[3], 2, 2)
histogram(model, a_episodes[4], a_tlabels[4], anoms[4], 3, 1)
histogram(model, a_episodes[5], a_tlabels[5], anoms[5], 3, 2)

subplot.update_layout(yaxis_type="log")
subplot.update_layout(yaxis2_type="log")
subplot.update_layout(yaxis3_type="log")
subplot.update_layout(yaxis4_type="log")
subplot.update_layout(yaxis5_type="log")
subplot.update_layout(yaxis6_type="log")
#print(subplot)
subplot.update_layout(margin=dict(l=5,b=0,r=5,t=20))
subplot.update_layout(dict(legend=dict(xanchor='center', y = -0.03, x=0.5, orientation='h')))


# Visualise Latent Space

In [ ]:
#assert config.latent_shape == 2 #otherwise... hmmm
print(config.latent_shape)

# Distance Graphs
Distance graphs are interactive and show the score (naturaly ordered) of each transition. 

In [ ]:
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu
import numpy as np
import datasets

def plot_distance(model, episode):
    z, d = distance(model, episode)
    images = T.HWC(tu.to_numpy(episode))

    plot = J.SimplePlot(np.arange(len(d)), d)
    image1 = J.SimpleImage(images[0])
    image2 = J.SimpleImage(images[1])

    def on_hover(trace, points, state):
        i = points.point_inds[0]
        image1.set_image(images[i])
        image2.set_image(images[i+1])

    plot.on_hover(on_hover)
    d_images = J.layout_horizontal(image1.fig, image2.fig)
    d_plot = J.layout_horizontal(plot.fig)
    J.display(J.layout_vertical(d_plot, d_images))

env = "Pong"
dataset = datasets.dataset('aad.raw.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
episode = [x for x in dataset.state.load(1)][0]
plot_distance(model, episode)



# Visualise Anomalies

In [ ]:
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu
import numpy as np

for i in range(len(a_episodes)):
    a = anoms[i][0]
    print(a)
    J.images(T.HWC(tu.to_numpy(a_episodes[i])))
    
    #J.scatter(np.arange(len(a_labels[i])), a_labels[i])
    

#### Load and show old 2D visualisation

In [ ]:
from pprint import pprint
from types import SimpleNamespace
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.wbutils as wbu
import datasets
import numpy as np

#load model
env = "Breakout"
run = "benedict-wilkins/anomapy/sssn-Breakout-20200209131829"
models, config = wbu.load(run)
config['state'] = dict(shape=config['state_shape'])
model = models['model.pt'].load(sssn.model(**config))
config = SimpleNamespace(**config)

#load data
num_episodes = 1
dataset = datasets.dataset('aad.raw.{0}'.format(env))
dataset.state.transform.to_float().CHW().torch()
episodes = [x for x in dataset.state.load(num_episodes)]



In [ ]:
#visualise
import anomapy.train.sssn as sssn
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.torchutils as tu

a = 5
episode = a_episodes[a]
labels = a_labels[a].astype(np.uint8)
print(labels)

def plot_latent(model, episode):
    z = tu.to_numpy(sssn.encode(model, episode))
    images = T.HWC(tu.to_numpy(episode))
    return J.scatter_image(z[:,0], z[:,1], images, scatter_colour=np.array(['blue','red'])[labels], line_colour='#b9d1fa', scale=1.5)

from ipywidgets import Image, Layout, VBox, HBox, interact, IntSlider, IntProgress, HTML, Output
import ipywidgets as widgets

fig, image = plot_latent(model, episode)

fig.update_layout(width=600, height=500)
#box_layout = widgets.Layout(display='flex',flex_flow='row',align_items='center',width='100%',height='100%')
#display(HBox([fig], layout=box_layout))

#box_layout = widgets.Layout(display='flex',flex_flow='row',align_items='center',width='100%',height='100%')
#display(HBox([fig, image_widget], layout=box_layout)) #basically... this needs to be done in jupyter..?!]


In [ ]:
image

-----------------
-----------------
-----------------
-----------------

# OTHER DEMOS


#### dynamic plot example

In [ ]:
import pyworld.toolkit.tools.visutils.jupyter as J
import time
import numpy as np

loss_plot = J.dynamic_plot(update_after=100)

for i in range(1000):
    time.sleep(0.01)
    loss_plot.update(i, np.sin(i/(np.pi*2)))
    

In [ ]:
import tensorflow as tf
z = tu.to_numpy(tf.keras.optimiser.encode(episode))
images = T.HWC(tu.to_numpy(episode))

#### Interactive plot example

In [ ]:
import pyworld.toolkit.tools.visutils.jupyter as J
import numpy as np

x = np.arange(1000)
y = np.sin(x)

plot = J.SimplePlot(x,y)
def on_hover(trace, points, state):
    ind = points.point_inds[0]
    print(ind)
    
plot.on_hover(on_hover)
plot.display()

#### Interactive plot with images

In [ ]:
import pyworld.toolkit.tools.visutils.jupyter as J
import numpy as np

N = 100
x = np.arange(N)
y = np.sin(x)
images = np.random.randint(0,255,size=(N, 100, 100, 3))
image = J.SimpleImage(images[0])

plot = J.SimplePlot(x,y)
def on_hover(trace, points, state):
    i = points.point_inds[0]
    image.set_image(images[i])
    
plot.on_hover(on_hover)

J.display(J.layout_horizontal(plot.fig, image.fig))



##### Sliding Histogram

In [ ]:
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.plot as vplot
import numpy as np

N = 100
x = np.arange(N)
y = np.sin(x)
yc = np.random.uniform(0,1,size=y.shape[0])

fig = J.histogram([y,yc], show=False)
vplot.histogram_slider(fig, sizes=np.arange(0.01,0.5,0.01))



#### subplots

In [ ]:
import numpy as np

N = 100
x = np.arange(N)
y = np.sin(x)
yc = np.random.uniform(0,1,size=y.shape[0])

import plotly
import plotly.graph_objects as go

def hist(x, color, name, showlegend=False):
    return go.Histogram(x=x, marker=dict(color=color), name=name, showlegend=showlegend)

c1 = 'red'
c2 = 'blue'


subplot = plotly.subplots.make_subplots(rows=3, cols=2)
trace1 = subplot.add_trace(hist(x, c1, 'anomaly', showlegend=True), row=1, col=1)
trace2 = subplot.add_trace(hist(y, c2, 'normal', showlegend=True), row=1, col=1)
trace3 = subplot.add_trace(hist(yc, c1, 'anomaly'), row=1, col=2)

subplot


### change scatter colour

In [ ]:
import pyworld.toolkit.tools.visutils.jupyter as J
import numpy as np

n = 10
x = np.arange(n)
y = np.sin(x)

plot = J.SimplePlot(x,y,mode=J.line_mode.marker)
def on_hover(trace, points, state):
    ind = points.point_inds[0]
    print(ind)
    
plot.on_hover(on_hover)
#print(plot.fig)
plot.fig.data[0]['marker'] = dict(color=np.random.randint(0,3,size=n).tolist())
plot.display()

